In [ ]:
#!!!!! 执行以下语句之前，切记，先确保库存stock_age_cal中的负数转换正数，然后添加至每日出库daily_stockout_sell,同时注意修改时间。
# 确保daily_stockout_sell_0930里面时间大于等于2018-11-30

In [1]:
import mysql.connector
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
from dateutil.parser import parse

In [2]:
# #数据库连接初始化设置
# mydb = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     passwd="qaz!@wsx",
#     database="work"
# )
# print(mydb)

In [3]:
#数据库连接初始化
mydb = mysql.connector.connect(
    host="rm-bp178mt2593im687xjo.mysql.rds.aliyuncs.com",
    user="root",
    passwd="KLxs8888",
    database="wdgj"
)
print(mydb)

In [4]:
#SQL语句
sqlStock='SELECT * FROM stock_age_cal'
sqlSell ='SELECT * FROM daily_stockout_sell'

In [5]:
#库存数据获取
mycursor=mydb.cursor()
mycursor.execute(sqlStock)
stockdata=mycursor.fetchall()

In [6]:
#将数据转化为DaraFrame
stockdata=pd.DataFrame(stockdata)
stockdata.columns=['GoodsNO','GoodsName','Total_in','date']
stockdata=stockdata[['GoodsNO','GoodsName','Total_in','date']]

In [7]:
stockdata.head()

,GoodsNO,GoodsName,Total_in,date
0,00329,爱斯博儿-考伊斯BCMP6006成人配件,1,2018-09-30
1,00342,Ttbaby贝仕达-雅拉牧场毛毯,1,2018-09-30
2,00439,蒂比-都灵婴儿车雨罩,0,2018-09-30
3,00442,FORBABY-P680婴儿车-金色车架,0,2018-09-30
4,00444,FORBABY-P680婴儿车-紫色遮阳棚,24,2018-09-30


In [8]:
stockdata =stockdata.fillna(np.nan)

In [9]:
stockdata.isnull().sum()

GoodsNO      0
GoodsName    0
Total_in     0
date         0
dtype: int64

In [10]:
stockdata['Total_in']=stockdata['Total_in'].fillna(0)

In [11]:
stockdata.shape

(138660, 4)

In [12]:
#删除小于等于0的数值
index = stockdata[stockdata['Total_in']<=0].index
stockdata=stockdata.drop(index=index,axis=0)

In [13]:
stockdata.shape

(130899, 4)

In [14]:
#按SKU分类汇总
stockdata_groupby = stockdata.groupby(['GoodsNO','GoodsName','date'],as_index=False)
stockdata_groupby=stockdata_groupby.aggregate(np.sum)

In [15]:
stockdata_groupby['cal_time']=''

In [16]:
stockdata_groupby.head()

,GoodsNO,GoodsName,date,Total_in,cal_time
0,.100135,蒂爱-折叠浴桶专属浴网,2019-01-23,1,
1,00329,爱斯博儿-考伊斯BCMP6006成人配件,2018-09-30,1,
2,00342,Ttbaby贝仕达-雅拉牧场毛毯,2018-09-30,1,
3,00442,FORBABY-P680婴儿车-金色车架,2018-10-05,9,
4,00444,FORBABY-P680婴儿车-紫色遮阳棚,2018-09-30,24,


In [17]:
stockdata_groupby.shape

(127018, 5)

In [18]:
#清除库存为0的数据
index = stockdata_groupby[stockdata_groupby['Total_in']==0].index
stockdata_groupby=stockdata_groupby.drop(index=index,axis=0)

In [19]:
stockdata_groupby.shape

(127018, 5)

In [20]:
stockdata_groupby.head()

,GoodsNO,GoodsName,date,Total_in,cal_time
0,.100135,蒂爱-折叠浴桶专属浴网,2019-01-23,1,
1,00329,爱斯博儿-考伊斯BCMP6006成人配件,2018-09-30,1,
2,00342,Ttbaby贝仕达-雅拉牧场毛毯,2018-09-30,1,
3,00442,FORBABY-P680婴儿车-金色车架,2018-10-05,9,
4,00444,FORBABY-P680婴儿车-紫色遮阳棚,2018-09-30,24,


In [21]:
#获取每日销量数据
mycursor=mydb.cursor()
mycursor.execute(sqlSell)
stocksell=mycursor.fetchall()

In [22]:
stocksell=pd.DataFrame(stocksell)
stocksell.columns=['SKU','name','total','date']
stocksell=stocksell[['SKU','name','total','date']]

In [23]:
stocksell.head()

,SKU,name,total,date
0,00329,爱斯博儿-考伊斯BCMP6006成人配件,1,2018-11-30
1,00499,蒂爱-克雷斯顿成人配件（肖恩）,5,2018-11-30
2,00499,蒂爱-克雷斯顿成人配件（肖恩）,1,2018-12-14
3,00518,蒂爱-3D摩丝经典版床垫（7cm）120*65,18,2018-11-30
4,00518,蒂爱-3D摩丝经典版床垫（7cm）120*65,1,2018-12-12


In [24]:
stocksell =stocksell.fillna(np.nan)

In [25]:
stocksell.isnull().sum()

SKU      0
name     0
total    0
date     0
dtype: int64

In [26]:
stocksell.shape

(167638, 4)

In [27]:
stocksell_groupby = stocksell.groupby(['SKU','name','date'],as_index=False)

In [28]:
stocksell_groupby

In [29]:
stocksell_groupby=stocksell_groupby.aggregate(np.sum)

In [30]:
stocksell_groupby.shape

(165781, 4)

In [31]:
index = stocksell_groupby[stocksell_groupby['date']<parse('2017-12-31').date()].index
stocksell_groupby=stocksell_groupby.drop(index=index,axis=0)

In [32]:
stocksell_groupby.head()

,SKU,name,date,total
0,00329,爱斯博儿-考伊斯BCMP6006成人配件,2018-11-30,1
1,00499,蒂爱-克雷斯顿成人配件（肖恩）,2018-11-30,5
2,00499,蒂爱-克雷斯顿成人配件（肖恩）,2018-12-08,2
3,00499,蒂爱-克雷斯顿成人配件（肖恩）,2018-12-14,1
4,00518,蒂爱-3D摩丝经典版床垫（7cm）120*65,2018-11-30,18


In [33]:
#按天分类
stocksell_groupby_date = stocksell_groupby.groupby(['date'],as_index=False)

In [34]:
stocksell_groupby_date

In [35]:
#按天获取对应数据
stocksell_groupby_date.get_group('2018-11-30').head()

,SKU,name,date,total
0,00329,爱斯博儿-考伊斯BCMP6006成人配件,2018-11-30,1
1,00499,蒂爱-克雷斯顿成人配件（肖恩）,2018-11-30,5
4,00518,蒂爱-3D摩丝经典版床垫（7cm）120*65,2018-11-30,18
20,00519,蒂爱-3D摩丝经典版床垫（7cm）120*60,2018-11-30,27
40,00521,蒂爱-3D摩丝经典版床垫（7cm）110*65,2018-11-30,1


In [36]:
# stockdata_groupby

In [37]:
#获取特定天的SKU，对应出库，构造SKU：Total字典
def get_sku_code(date):
    try:
        df=stocksell_groupby_date.get_group(date)
    except:
        df=[]
    l=len(df)
    dict_new={}
    for i in range(l):
        dict_new[df['SKU'].iloc[i]]=df['total'].iloc[i]
    return dict_new

In [38]:
#计算库存变化，按照先进先出原则
def sku_stock_delete(time,stock_change):
    sell_dict=get_sku_code(time)#获取特定某一天的SKU:Total
    total=0
    a=0
    print(len(sell_dict))
    for key in sell_dict:
        total+=1
#         print(total)
        sell=sell_dict[key]
        #获取满足条件的对应所在行的index
        stock_index=stock_change[(stock_change['date']<=parse(time).date())&(stock_change['GoodsNO']==key)\
                                     &(stock_change['Total_in']>0)].sort_values(by ='date',ascending=True)['Total_in'].index
        #遍历每个index，按条件扣减对应的库存
        for i in stock_index:
            stock=stock_change['Total_in'].loc[i]
            if sell>=stock:
                stock_change['Total_in'].loc[i]=0
                sell=sell-stock
#                 stock_change['stock_time'].loc[i]=time
            else:
                stock_change['Total_in'].loc[i]=stock-sell
                sell=0
#                 stock_change['stock_time'].loc[i]=time
                break
        a=a+sell
    print(total)
    index_0 = stock_change[stock_change['Total_in']==0].index
    stock_change=stock_change.drop(index=index_0,axis=0)
    return stock_change,a

In [39]:
#重新创建一个DataFrame，复制一份stockdata_groupby,用来确保在调用函数的过程中不改变stockdata_groupby
stock_change_data=pd.DataFrame()
stock_change_data=pd.concat([stock_change_data,stockdata_groupby])

In [40]:
#构造存放计算后每天的库存数据表格
stock_age_cal=pd.DataFrame()
t=0

In [41]:
starttime = datetime.now()
startdate=date(2018,11,30)
for i in range(91):
    start=startdate + timedelta(i)
    print(start)
    stock_change=sku_stock_delete(str(start),stock_change_data)#调用函数计算库存占用
    stock_change[0]['cal_time']=start#标记库存占用计算日期
    stock_age_cal=pd.concat([stock_age_cal,stock_change[0][stock_change[0]['date']<=start-timedelta(60)]])#筛选60天前的数据
    t+=stock_change[1]
endtime=datetime.now()
print ((endtime - starttime).seconds/60/60)

2018-11-30
6718


D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


6718
2018-12-01
2063
2063
2018-12-02
1832
1832
2018-12-03
2004
2004
2018-12-04
2207
2207
2018-12-05
2014
2014
2018-12-06
2077
2077
2018-12-07
273
273
2018-12-08
2637
2637
2018-12-09
878
878
2018-12-10
1189
1189
2018-12-11
859
859
2018-12-12
3355
3355
2018-12-13
3429
3429
2018-12-14
3225
3225
2018-12-15
3040
3040
2018-12-16
3283
3283
2018-12-17
1632
1632
2018-12-18
1520
1520
2018-12-19
2914
2914
2018-12-20
2618
2618
2018-12-21
2598
2598
2018-12-22
2217
2217
2018-12-23
1900
1900
2018-12-24
2063
2063
2018-12-25
2053
2053
2018-12-26
2014
2014
2018-12-27
1971
1971
2018-12-28
2104
2104
2018-12-29
2016
2016
2018-12-30
911
911
2018-12-31
2548
2548
2019-01-01
1914
1914
2019-01-02
2175
2175
2019-01-03
1975
1975
2019-01-04
2041
2041
2019-01-05
1986
1986
2019-01-06
2063
2063
2019-01-07
2174
2174
2019-01-08
2221
2221
2019-01-09
2013
2013
2019-01-10
1689
1689
2019-01-11
980
980
2019-01-12
799
799
2019-01-13
3166
3166
2019-01-14
3225
3225
2019-01-15
2660
2660
2019-01-16
2696
2696
2019-01-17
2713
2713

In [42]:
stock_age_cal.to_csv('stock_age_cal.csv',header=True,index=False)

In [43]:
stock_change_data.to_csv('stock_change_data.csv',header=True,index=False)

In [44]:
stock_age_cal=pd.read_csv('stock_age_cal.csv',header = 0)

In [45]:
stock_age_cal.head()

,GoodsNO,GoodsName,date,Total_in,cal_time
0,00342,Ttbaby贝仕达-雅拉牧场毛毯,2018-09-30,1,2018-11-30
1,00444,FORBABY-P680婴儿车-紫色遮阳棚,2018-09-30,24,2018-11-30
2,00499,蒂爱-克雷斯顿成人配件（肖恩）,2018-09-30,169,2018-11-30
3,00518,蒂爱-3D摩丝经典版床垫（7cm）120*65,2018-09-30,271,2018-11-30
4,00519,蒂爱-3D摩丝经典版床垫（7cm）120*60,2018-09-30,196,2018-11-30


In [46]:
#构造新的表格复制一份stock_age_cal
stock_age_cal_copy=pd.DataFrame()
stock_age_cal_copy=pd.concat([stock_age_cal_copy,stock_age_cal])

In [ ]:
# stock_age_cal.head()

In [ ]:
# stock_age_cal1=stock_age_cal1[stock_age_cal1['cal_time']<'2018-02-11']

In [ ]:
# stock_age_cal_copy=pd.concat([stock_age_cal1,stock_age_cal])

In [ ]:
# stock_age_cal_copy=stock_age_cal_copy[stock_age_cal_copy['cal_time']<='2018-12-31']

In [ ]:
# stock_age_cal_copy.to_csv('stock_age_cal12.csv',header=True,index=False)

In [47]:
stock_age_cal_copy['date']=pd.to_datetime(stock_age_cal_copy['date'])
stock_age_cal_copy['cal_time']=pd.to_datetime(stock_age_cal_copy['cal_time'])

In [48]:
stock_age_cal_copy['stockAge']=stock_age_cal_copy['cal_time']-stock_age_cal_copy['date']

In [49]:
stock_age_cal_copy.head()

,GoodsNO,GoodsName,date,Total_in,cal_time,stockAge
0,00342,Ttbaby贝仕达-雅拉牧场毛毯,2018-09-30,1,2018-11-30,61 days
1,00444,FORBABY-P680婴儿车-紫色遮阳棚,2018-09-30,24,2018-11-30,61 days
2,00499,蒂爱-克雷斯顿成人配件（肖恩）,2018-09-30,169,2018-11-30,61 days
3,00518,蒂爱-3D摩丝经典版床垫（7cm）120*65,2018-09-30,271,2018-11-30,61 days
4,00519,蒂爱-3D摩丝经典版床垫（7cm）120*60,2018-09-30,196,2018-11-30,61 days


In [50]:
#选择库龄大于60天的SKU
stock_age_cal_copy_60=stock_age_cal_copy[(stock_age_cal_copy['stockAge']>=timedelta(60))&(stock_age_cal_copy['cal_time']>'2018-11-30')]

In [51]:
stock_age_cal_copy_60.head()

,GoodsNO,GoodsName,date,Total_in,cal_time,stockAge
6453,00342,Ttbaby贝仕达-雅拉牧场毛毯,2018-09-30,1,2018-12-01,62 days
6454,00444,FORBABY-P680婴儿车-紫色遮阳棚,2018-09-30,24,2018-12-01,62 days
6455,00499,蒂爱-克雷斯顿成人配件（肖恩）,2018-09-30,169,2018-12-01,62 days
6456,00518,蒂爱-3D摩丝经典版床垫（7cm）120*65,2018-09-30,271,2018-12-01,62 days
6457,00519,蒂爱-3D摩丝经典版床垫（7cm）120*60,2018-09-30,195,2018-12-01,62 days


In [52]:
# #获取商品固定价格
# goodslist=pd.read_csv('C:\\Users\\Administrator\\PycharmProjects\\test\\g_goods_goodslist.csv',\
#                       header = 0,dtype=object,engine='python',encoding='utf-8')
# goodslist_short=goodslist[['GoodsNO','FixCostPrice']]
# goodslist_short=goodslist_short.drop_duplicates(["GoodsNO"]) #SKU去重

In [53]:
mydb = mysql.connector.connect(
    host="rm-bp178mt2593im687xjo.mysql.rds.aliyuncs.com",
    user="root",
    passwd="KLxs8888",
    database="wdgj"
)

In [54]:
sql_goodslist='SELECT GoodsNO,FixCostPrice FROM g_goods_goodslist'
mycursor=mydb.cursor()
mycursor.execute(sql_goodslist)
goodslist=mycursor.fetchall()

In [55]:
goodslist = pd.DataFrame(goodslist)

In [56]:
goodslist.columns=['GoodsNO','FixCostPrice']

In [57]:
goodslist = pd.DataFrame(goodslist)
goodslist_short=goodslist[['GoodsNO','FixCostPrice']]
goodslist_short=goodslist_short.drop_duplicates(["GoodsNO"])

In [58]:
#匹配库存和商品列表
data_merge=pd.merge(stock_age_cal_copy_60,goodslist_short,on=['GoodsNO'],how='left')
data_merge['FixCostPrice']=data_merge['FixCostPrice'].apply(pd.to_numeric) # 将FixCostPrice转换成数值类型

In [59]:
data_merge=data_merge.fillna(np.nan)

In [60]:
data_merge.isnull().sum()

GoodsNO           0
GoodsName         0
date              0
Total_in          0
cal_time          0
stockAge          0
FixCostPrice    486
dtype: int64

In [61]:
data_merge['FixCostPrice']=data_merge['FixCostPrice'].fillna(0)

In [62]:
#计算每个SKU的库存占用费
data_merge['stockDelayCost']=data_merge['Total_in']*data_merge['FixCostPrice']*0.05/30

In [63]:
data_merge=data_merge[(data_merge['stockAge']>=timedelta(60))&(data_merge['cal_time']>'2018-12-31')]

In [64]:
data_merge.to_excel('data_merge.xlsx',header=True,index=False)

In [65]:
data_merge['cal_time_month']=data_merge.cal_time.map(lambda x: x.strftime('%Y-%m')) # 取月份数据

In [66]:
#分SKU，月份汇总
data_merge_groupby=data_merge.groupby(['GoodsNO','GoodsName','cal_time_month'],as_index=False)
data_merge_groupby=data_merge_groupby.aggregate(np.sum)

In [67]:
data_merge_groupby.head()

,GoodsNO,GoodsName,cal_time_month,Total_in,FixCostPrice,stockDelayCost
0,00342,Ttbaby贝仕达-雅拉牧场毛毯,2019-01,31,1302.0,2.170
1,00342,Ttbaby贝仕达-雅拉牧场毛毯,2019-02,28,1176.0,1.960
2,00442,FORBABY-P680婴儿车-金色车架,2019-01,279,14911.0,223.665
3,00442,FORBABY-P680婴儿车-金色车架,2019-02,252,13468.0,202.020
4,00444,FORBABY-P680婴儿车-紫色遮阳棚,2019-01,744,651.0,26.040


In [68]:
#获取小组信息并去重
GoodsInfo =pd.read_csv('C:\\Users\\Administrator\\PycharmProjects\\test\\goodsinfo0305.csv',header = 0,engine='python')
GoodsInfo.columns=['GoodsNO','GoodsName','barCode','Company','Dept','tamllTax','Group','Holder','GradeTwo','GradeThird','GradeForth','SPU']
GoodsInfoShort=GoodsInfo[['GoodsNO','Company','Dept']]
GoodsInfoShort=GoodsInfoShort.drop_duplicates(['GoodsNO'])

In [69]:
#匹配部门
data_merge_dept=pd.merge(data_merge_groupby,GoodsInfoShort,on=['GoodsNO'],how='left')

In [70]:
data_merge_dept.columns

Index(['GoodsNO', 'GoodsName', 'cal_time_month', 'Total_in', 'FixCostPrice',
       'stockDelayCost', 'Company', 'Dept'],
      dtype='object')

In [71]:
data_merge_dept=data_merge_dept.fillna(np.nan)
data_merge_dept.isnull().sum()

GoodsNO             0
GoodsName           0
cal_time_month      0
Total_in            0
FixCostPrice        0
stockDelayCost      0
Company           143
Dept              143
dtype: int64

In [72]:
data_merge_groupby.columns

Index(['GoodsNO', 'GoodsName', 'cal_time_month', 'Total_in', 'FixCostPrice',
       'stockDelayCost'],
      dtype='object')

In [73]:
data_merge_dept=data_merge_dept[['GoodsNO', 'GoodsName', 'cal_time_month', 'Total_in', 'stockDelayCost','Company', 'Dept']]

In [74]:
data_merge_dept.to_excel('data_merge_dept.xlsx',header=True,index=False)

In [75]:
#按月，公司，部门汇总
data_merge_dept_gb = data_merge_dept.groupby(['Company','Dept','cal_time_month'],as_index=False)
data_merge_dept_gb=data_merge_dept_gb.aggregate(np.sum)
data_merge_dept_gb=data_merge_dept_gb[['Company','Dept','cal_time_month','stockDelayCost']]

In [76]:
data_merge_dept_gb.head()

,Company,Dept,cal_time_month,stockDelayCost
0,可优比母婴用品有限公司,KUB出行,2019-01,278878.608427
1,可优比母婴用品有限公司,KUB出行,2019-02,274349.585267
2,可优比母婴用品有限公司,KUB包材,2019-01,38685.986413
3,可优比母婴用品有限公司,KUB包材,2019-02,50950.050425
4,可优比母婴用品有限公司,KUB喂辅,2019-01,503729.020616


In [77]:
data_merge_dept_gb.to_excel("2019库存占用带小组6.xlsx",header=True,index=False)

In [ ]:
# stocksell_groupby[stocksell_groupby['date']<=parse('2018-12-31').date()].sort_values(by ='date',ascending=True).head()

In [78]:
#校核检验结果
result=stockdata_groupby[stockdata_groupby['date']<=parse('2019-02-28').date()].groupby(['date'],as_index=False)
result=result.aggregate(np.sum)#初始库存总数
result1=stock_change_data[stock_change_data['date']<=parse('2019-02-28').date()].groupby(['date'],as_index=False)
result1=result1.aggregate(np.sum)#最终库存总数
result2=stocksell_groupby[stocksell_groupby['date']<=parse('2019-02-28').date()].groupby(['date'],as_index=False)
result2=result2.aggregate(np.sum)#出库总数
result.Total_in.sum()

226172455

In [79]:
#检验是否计算正确，为零则表示准确无误
result1.Total_in.sum()-(result.Total_in.sum()-result2.total.sum()+t)

0

In [ ]:
result1.Total_in.sum()

In [ ]:
result2.total.sum()

In [ ]:
t

In [ ]:
startdate=date(2018,11,30)
for i in range(91):
    start=startdate + timedelta(i)
    print(start)